In [ ]:
import os
import numpy as np
import cv2
import time
import tensorflow
from tensorflow import keras
from keras import callbacks
from keras import layers, models, regularizers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# giving directory path to data set
dataset_path='/content/drive/MyDrive/train'

In [ ]:
image_size=(65,65)
# data_generator = ImageDataGenerator(rescale=1./255)
data_generator=ImageDataGenerator(rescale=1./255,rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.2)
train_dataset=data_generator.flow_from_directory(dataset_path,target_size=image_size,batch_size=32,class_mode='binary',shuffle=True,subset="training")
validation_dataset=data_generator.flow_from_directory(dataset_path,target_size=image_size,batch_size=32,class_mode='binary',shuffle=True,subset="validation")

In [ ]:
train_dataset.class_indices

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',padding="same"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu',padding="same"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',padding="same"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
earlystopping=callbacks.EarlyStopping(monitor="accuracy",mode="max",patience=3,restore_best_weights=True)

In [ ]:
model.fit(train_dataset,epochs=15,batch_size=32,validation_data=validation_dataset,callbacks=[earlystopping])

In [ ]:
# Load the Haar cascade classifiers for face and eye detection. You may download it from OpenCV2 github repo
face=cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
eye=cv2.CascadeClassifier('/content/haarcascade_eye_tree_eyeglasses.xml')

In [ ]:
def duration():
    start_time=time.time()
    end_time=start_time+0.7
    open=0
    closed=0
    #This loop runs via function for 0.7 second as it is equal to twice the average required for human to blink. If the eye is closed for more than
    #time person is possibly drowsie.
    while(time.time()<end_time):
        ret,frame=video_capture.read()
        if(ret==True):
            gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)   # Convert the frame to grayscale as model is trained on gray scale images
            faces=face.detectMultiScale(gray)             # Detect faces
            for (x,y,w,h) in faces:                       # Iterate over the detected faces
                roi_gray=gray[y:y+h,x:x+w]                # Get the region of interest containing the face in gray scale image
                roi_color=frame[y:y+h,x:x+w]              # Get the region of interest containing the face in colored image
                eyes=eye.detectMultiScale(roi_gray)       # Detect eyes in the ROI
                for (a,b,c,d) in eyes:                    #Iterate over the detected faces
                    eye_roi_gray=roi_gray[b:b+d,a:a+c]    # Get the region of interest containing the eye in grey scale image
                    eye_roi_color=roi_color[b:b+d,a:a+c]  # Get the region of interest containing the eye in colored image
                    eye_image=eye_roi_gray.copy()         #Creating an image out of the eye_roi_grey for further process
                    #Processing the image according to requirments of the model
                    resized_frame=cv2.resize(eye_image,(65,65))
                    normalized_frame=resized_frame/255.0
                    flattened_frame=normalized_frame.flatten()
                    #Predicting eyes condition open or closed
                    predictions=model.predict(flattened_frame)
                    if(predictions==0):
                        closed=closed+1
                    elif(predictions==1):
                        open=open+1
    return open,closed

In [ ]:

video_capture=cv2.VideoCapture(0)
if(video_capture.isOpened()):
    #This loop runs for the time the vehicle is on. It can beforcefully shut by pressing 'q' or any command as possible. The manufacturer
    #can apply the optimal condition
    while(True):
        open,closed=duration(open,closed)
        if(open>=1.05*closed):
            print("Normal")               # Normal.No action required
        elif(open>=0.3*closed):
            print("notification")         #Insert a code to issue a notificatio or confirmation prompt
        elif(open<3*closed):
            print("alarm")                #Insert a code to ring an alarm

        #The above choosen value are custom and calculated by self estimation.
        #This can be changed according to users preference
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
else:
    print("Camera is closed. Try giving other camera index in videocapture")
video_capture.release()